In [73]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [74]:
data=pd.read_csv('/content/AAPL_Price_History_20211207.csv')

In [75]:
data.head()

,Date,Open,High,Low,Close,Volume,Unnamed: 6
0,12/7/2011,13.9261,13.9621,13.8129,13.8961,305000048.0,NaN
1,12/8/2011,13.9803,14.1250,13.9368,13.9521,376356580.0,NaN
2,12/9/2011,14.0303,14.0728,13.9653,14.0578,297143408.0,NaN
3,12/12/2011,13.9886,14.0678,13.9089,13.9943,301065820.0,NaN
4,12/13/2011,14.0357,14.1214,13.8250,13.8861,339137092.0,NaN


In [76]:
data.dtypes

Date           object
Open          float64
High          float64
Low           float64
Close         float64
Volume        float64
Unnamed: 6    float64
dtype: object

In [77]:
data.nunique()

Date          2517
Open          2406
High          2401
Low           2409
Close         2415
Volume        2517
Unnamed: 6       0
dtype: int64

In [78]:
data=data.drop('Unnamed: 6', axis=1)

In [79]:
data.isnull()

,Date,Open,High,Low,Close,Volume
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
2515,False,False,False,False,False,False
2516,False,False,False,False,False,False
2517,True,True,True,True,True,True
2518,True,True,True,True,True,True


In [80]:
data.dropna(inplace=True)


In [81]:
# Summary Statistics for all Variables
data.describe().round(2)

,Open,High,Low,Close,Volume
count,2517.00,2517.00,2517.00,2517.00,2.517000e+03
mean,47.92,48.42,47.44,47.95,2.164128e+08
std,37.36,37.80,36.94,37.40,1.758237e+08
min,13.58,13.70,13.49,13.53,4.099995e+07
25%,23.50,23.68,23.25,23.48,1.035447e+08
50%,32.09,32.34,31.85,32.14,1.532756e+08
75%,52.72,53.41,52.32,52.63,2.677400e+08
max,169.08,171.58,168.34,171.18,1.506113e+09


In [82]:
# Calculate the complete correlation matrix
data.corr()

,Open,High,Low,Close,Volume
Open,1.000000,0.999882,0.999840,0.999721,-0.419466
High,0.999882,1.000000,0.999801,0.999857,-0.417482
Low,0.999840,0.999801,1.000000,0.999858,-0.422621
Close,0.999721,0.999857,0.999858,1.000000,-0.420279
Volume,-0.419466,-0.417482,-0.422621,-0.420279,1.000000


In [83]:
#from datetime import datetime
#data.Date=pd.to_datetime(data.Date)
#data.info()
#data['Month']=pd.DatetimeIndex(data['Date']).month
#data['Year']=pd.DatetimeIndex(data['Date']).year
#data['tday']=pd.DatetimeIndex(data['Date']).day
data.head()

,Date,Open,High,Low,Close,Volume
0,12/7/2011,13.9261,13.9621,13.8129,13.8961,305000048.0
1,12/8/2011,13.9803,14.1250,13.9368,13.9521,376356580.0
2,12/9/2011,14.0303,14.0728,13.9653,14.0578,297143408.0
3,12/12/2011,13.9886,14.0678,13.9089,13.9943,301065820.0
4,12/13/2011,14.0357,14.1214,13.8250,13.8861,339137092.0


In [84]:
#data.drop({'Date'},inplace=True,axis='columns')
#data.head()

In [85]:
import math
train_len = math.ceil(len(data)*.8)
train_len

2014

In [86]:
data1 = data

In [87]:
data = data.filter(['Close'])

In [88]:
datas = data.values
len(datas)

2517

In [89]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(datas)

In [90]:
train = scaled_data[0: train_len,:]

In [91]:
X_train = []
y_train = []

for i in range (60 ,len(train)):
  X_train.append(train[i-60:i,0])
  y_train.append(train[i,0])

In [92]:
X_train, y_train = np.array(X_train ), np.array(y_train)

In [93]:
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_train.shape

(1954, 60, 1)

In [94]:
model = Sequential()
model.add(LSTM(50 , return_sequences=True , input_shape = (X_train.shape[1],1)))
model.add(LSTM(50 , return_sequences=False))
model.add(Dense(30))
model.add(Dense(1))

In [95]:
model.compile(optimizer='adam',loss = 'mean_squared_error', metrics= 'accuracy')

In [96]:
model.fit(X_train,y_train,batch_size=1 , epochs=2)

Epoch 1/2
1954/1954 [==============================] - 77s 38ms/step - loss: 2.4227e-04 - accuracy: 0.0000e+00
Epoch 2/2
1954/1954 [==============================] - 67s 35ms/step - loss: 9.0411e-05 - accuracy: 0.0000e+00


In [97]:
len(datas)

2517

In [98]:
len(train)

2014

In [27]:
#len(test)

In [99]:
test = scaled_data[train_len-60:,:]

In [47]:
len(test)

60

In [108]:
X_test = []
y_test = datas[train_len:, :]
for i in range(60,len(test)):
  X_test.append(test[i-60:i,0])

In [109]:
X_test

[array([0.26287565, 0.26414431, 0.26742698, 0.26455663, 0.25943441,
        0.26100437, 0.25935511, 0.26466764, 0.26285979, 0.26116296,
        0.26932997, 0.27031318, 0.26138497, 0.26433461, 0.27415088,
        0.27423018, 0.27001187, 0.2741826 , 0.27903523, 0.2887405 ,
        0.28820132, 0.28732911, 0.28582258, 0.28726568, 0.28905766,
        0.29555956, 0.29468735, 0.29979372, 0.30042805, 0.30518553,
        0.30910252, 0.29996816, 0.29992058, 0.30864263, 0.31983858,
        0.32250277, 0.32191601, 0.32209045, 0.32556341, 0.32668935,
        0.32995616, 0.32957556, 0.33355598, 0.33065392, 0.3356017 ,
        0.33772671, 0.33644219, 0.33152612, 0.32965485, 0.32929011,
        0.33656906, 0.33327053, 0.33890022, 0.33796458, 0.33306438,
        0.32559513, 0.32922667, 0.33531625, 0.34345155, 0.33744126]),
 array([0.26414431, 0.26742698, 0.26455663, 0.25943441, 0.26100437,
        0.25935511, 0.26466764, 0.26285979, 0.26116296, 0.26932997,
        0.27031318, 0.26138497, 0.26433461, 0.

In [111]:
X_test = np.array(X_test)

In [112]:
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1)) 

In [113]:
pred = model.predict(X_test)
pred = scaler.inverse_transform(pred)

In [114]:
from sklearn.metrics import r2_score
print(r2_score(pred,y_test))

0.967887205885719
